# How to use the R package `arules` from Python using `arulespy`

This document is also avaialble as a IPython notebook: https://github.com/mhahsler/arulespy/blob/main/examples/arules.ipynb


## Installation 

The package can be installed using pip.

```
pip install arulespy
```

The following may be necessary on Windows to set the 'R_HOME' for `rpy2` correctly:

In [1]:
# from rpy2 import situation
# import os
#
# os.environ['R_HOME'] = situation.r_home_from_registry()
# situation.get_r_home()

## Examples

Import the `arules` module from package `arulespy`.

In [2]:
from arulespy.arules import Transactions, apriori, parameters, concat

### Creating transaction data

The data need to be prepared as a Pandas dataframe. Here we have 9 transactions with three items called A, B and C. True means that a transaction contains the item.

In [3]:
import pandas as pd

df = pd.DataFrame (
    [
        [True,True, True],
        [True, False,False],
        [True, True, True],
        [True, False, False],
        [True, True, True],
        [True, False, True],
        [True, True, True],
        [False, False, True],
        [False, True, True],
        [True, False, True],
    ],
    columns=list ('ABC')) 

df

,A,B,C
0,True,True,True
1,True,False,False
2,True,True,True
3,True,False,False
4,True,True,True
5,True,False,True
6,True,True,True
7,False,False,True
8,False,True,True
9,True,False,True


Convert the pandas dataframe into a sparse transactions object.

In [4]:
trans = Transactions.from_df(df)
print(trans)

trans.as_df()

transactions in sparse format with
 10 transactions (rows) and
 3 items (columns)



,items,transactionID
1,"{A,B,C}",0
2,{A},1
3,"{A,B,C}",2
4,{A},3
5,"{A,B,C}",4
6,"{A,C}",5
7,"{A,B,C}",6
8,{C},7
9,"{B,C}",8
10,"{A,C}",9


We can calculate item frequencies, sample transactions or remove duplicate transactions. All available functions can be found at the end of this document.

In [5]:
trans.itemFrequency(type = 'relative')

array([0.8, 0.5, 0.8])

In [6]:
trans.sample(3).as_df()

,items,transactionID
10,"{A,C}",9
4,{A},3
7,"{A,B,C}",6


In [7]:
trans.unique().as_df()

,items,transactionID
1,"{A,B,C}",0
2,{A},1
6,"{A,C}",5
8,{C},7
9,"{B,C}",8


Create new data that uses the same encoding as an existing transaction set. Note that the following dataframe
has the columns (items) in reverse order and `transactions()` copies the itemcoding from `trans`. 

In [8]:
df2 = pd.DataFrame (
    [
        [True,True, False]
    ],
    columns=list ('CBA')) 

df2

,C,B,A
0,True,True,False


In [9]:
Transactions.from_df(df2, trans).as_df()

,items,transactionID
1,"{B,C}",0


Add the new transaction to the existing transactions.

In [10]:
concat([trans, Transactions.from_df(df2, trans)]).as_df()

,items,transactionID
1,"{A,B,C}",0
2,{A},1
3,"{A,B,C}",2
4,{A},3
5,"{A,B,C}",4
6,"{A,C}",5
7,"{A,B,C}",6
8,{C},7
9,"{B,C}",8
10,"{A,C}",9


Converting a dataframe with nominal and numeric variables. The nominal variables are converted into the form `variable=value` and
numeric variables are first discretized (see `arules.discretizeDF()`).

In [11]:
df2 = pd.DataFrame (
    [
        ['red',  12, True],
        ['blue', 10, False],
        ['red',  18, True],
        ['green',18, False],
        ['red',  16, True],
        ['blue',  9, False]
    ],
    columns=list(['color', 'size', 'class'])) 

trans2 = Transactions.from_df(df2)
trans2.as_df()

,items,transactionID
1,"{color=red,size=[11.3,16.7),class}",0
2,"{color=blue,size=[9,11.3)}",1
3,"{color=red,size=[16.7,18],class}",2
4,"{color=green,size=[16.7,18]}",3
5,"{color=red,size=[11.3,16.7),class}",4
6,"{color=blue,size=[9,11.3)}",5


Details on item label creation can be retrieved using `arules.itemInfo()`.

In [12]:
trans2.itemInfo()

,labels,variables,levels
1,color=blue,color,blue
2,color=green,color,green
3,color=red,color,red
4,"size=[9,11.3)",size,"[9,11.3)"
5,"size=[11.3,16.7)",size,"[11.3,16.7)"
6,"size=[16.7,18]",size,"[16.7,18]"
7,class,class,TRUE


## Mine association rules

`arules.apriori()` calls the apriori algorithm and converts the results into a Python `arulespy.arules.Rules` object. Parameters for the algorithm
are specified as `dict` inside the `arules.parameter()` funcition.

In [13]:
rules = apriori(trans,
                    parameter = parameters({"supp": 0.1, "conf": 0.8}), 
                    control = parameters({"verbose": False}))  


rules.as_df()

,LHS,RHS,support,confidence,coverage,lift,count
1,{},{A},0.8,0.8,1.0,1.00,8
2,{},{C},0.8,0.8,1.0,1.00,8
3,{B},{A},0.4,0.8,0.5,1.00,4
4,{B},{C},0.5,1.0,0.5,1.25,5
5,"{A,B}",{C},0.4,1.0,0.4,1.25,4
6,"{B,C}",{A},0.4,0.8,0.5,1.00,4


In [14]:
rules.quality()


,support,confidence,coverage,lift,count
1,0.8,0.8,1.0,1.00,8
2,0.8,0.8,1.0,1.00,8
3,0.4,0.8,0.5,1.00,4
4,0.5,1.0,0.5,1.25,5
5,0.4,1.0,0.4,1.25,4
6,0.4,0.8,0.5,1.00,4


Python-style `len()` and slicing is available.

In [15]:
len(rules)

6

In [16]:
rules[0:3].as_df()

,LHS,RHS,support,confidence,coverage,lift,count
1,{},{A},0.8,0.8,1.0,1.0,8
2,{},{C},0.8,0.8,1.0,1.0,8
3,{B},{A},0.4,0.8,0.5,1.0,4


In [17]:
rules[[True, False, True, False, True, False]].as_df()

,LHS,RHS,support,confidence,coverage,lift,count
1,{},{A},0.8,0.8,1.0,1.00,8
3,{B},{A},0.4,0.8,0.5,1.00,4
5,"{A,B}",{C},0.4,1.0,0.4,1.25,4


## Accessing Rules

rules can be converted into various Python data structures. 

In [18]:
rules.labels()

['{} => {A}',
 '{} => {C}',
 '{B} => {A}',
 '{B} => {C}',
 '{A,B} => {C}',
 '{B,C} => {A}']

In [19]:
rules.items().as_df()

,items
1,{A}
2,{C}
3,"{A,B}"
4,"{B,C}"
5,"{A,B,C}"
6,"{A,B,C}"


In [20]:
rules.lhs().as_df()

,items
1,{}
2,{}
3,{B}
4,{B}
5,"{A,B}"
6,"{B,C}"


In [21]:
rules.lhs().as_list()

[[], [], ['B'], ['B'], ['A', 'B'], ['B', 'C']]

In [22]:
rules.rhs().as_df()

,items
1,{A}
2,{C}
3,{A}
4,{C}
5,{C}
6,{A}


In [23]:
rules.sort(by = 'lift').as_df()

,LHS,RHS,support,confidence,coverage,lift,count
4,{B},{C},0.5,1.0,0.5,1.25,5
5,"{A,B}",{C},0.4,1.0,0.4,1.25,4
1,{},{A},0.8,0.8,1.0,1.00,8
2,{},{C},0.8,0.8,1.0,1.00,8
3,{B},{A},0.4,0.8,0.5,1.00,4
6,"{B,C}",{A},0.4,0.8,0.5,1.00,4


## Work With Interest Measures

Interest measures are stored as the quality attribute in rules and itemsets.

In [24]:
rules.quality()

,support,confidence,coverage,lift,count
1,0.8,0.8,1.0,1.00,8
2,0.8,0.8,1.0,1.00,8
3,0.4,0.8,0.5,1.00,4
4,0.5,1.0,0.5,1.25,5
5,0.4,1.0,0.4,1.25,4
6,0.4,0.8,0.5,1.00,4


Additional interest measures can be calculated with `interestMeasure()` and added to rules or itemsets using `addQuality()`. See all [available meassures](https://mhahsler.github.io/arules/docs/measures). To calculate some measures, transactions need to
be specified.

In [25]:
im = rules.interestMeasure(["phi", 'support'])
im

,phi,support
1,NaN,0.8
2,NaN,0.8
3,0.000000,0.4
4,0.500000,0.5
5,0.408248,0.4
6,0.000000,0.4


In [26]:
rules.addQuality(im)
rules.as_df()

,LHS,RHS,support,confidence,coverage,lift,count,phi
1,{},{A},0.8,0.8,1.0,1.00,8,NaN
2,{},{C},0.8,0.8,1.0,1.00,8,NaN
3,{B},{A},0.4,0.8,0.5,1.00,4,0.000000
4,{B},{C},0.5,1.0,0.5,1.25,5,0.500000
5,"{A,B}",{C},0.4,1.0,0.4,1.25,4,0.408248
6,"{B,C}",{A},0.4,0.8,0.5,1.00,4,0.000000


## Filter Redundant Rules

In [27]:
rules[[not x for x in rules.is_redundant()]].as_df()

,LHS,RHS,support,confidence,coverage,lift,count,phi
1,{},{A},0.8,0.8,1.0,1.00,8,NaN
2,{},{C},0.8,0.8,1.0,1.00,8,NaN
4,{B},{C},0.5,1.0,0.5,1.25,5,0.5


In [28]:
rules.is_redundant()


0,0,1,0,1,1


Find maximal rules.

In [29]:
rules.is_maximal()

0,0,0,0,1,1


## Help for Functions Available via arulespy

In [30]:
help(apriori)

Help on function wrapper in module arulespy.arules:

wrapper(*args, **kwargs)
    Wrapper around an R function.
    
    The docstring below is built from the R documentation.
    
    description
    -----------
    
    
     Mine frequent itemsets, association rules or association hyperedges using
     the Apriori algorithm.
     
    
    
    apriori(
        data,
        parameter = rinterface.NULL,
        appearance = rinterface.NULL,
        control = rinterface.NULL,
        ___ = (was "..."). R ellipsis (any number of parameters),
    )
    
    Args:
       data :  object of class transactions. Any data structure which can be
      coerced into transactions (e.g., a binary matrix, a
      data.frame or a tibble) can also be specified and will be
      internally coerced to transactions.
    
       parameter :  object of class APparameter or named list.  The default
      behavior is to mine rules with minimum support of 0.1,
      minimum confidence of 0.8, maximum of 10 

## Low-level R arules interface

arules functions can also be directly called using
`R.<arules R function>()`. The result will be a `rpy2` data type.
Transactions, itemsets and rules can manually be converted to Python
classes using.

In [31]:
from arulespy.arules import R, Itemsets, a2py

In [32]:
help(R.random_patterns)

Help on DocumentedSTFunction in module rpy2.robjects.functions:

<rpy2.robjects.functions.DocumentedSTFunction ob...32885c0> [RTYPES.CLOSXP]
R classes: ('function',)
    Wrapper around an R function.
    
    The docstring below is built from the R documentation.
    
    description
    -----------
    
    
     Simulate random  transactions  using different methods.
     
    
    
    random.patterns(
        nItems,
        nPats = 2000.0,
        method = rinterface.NULL,
        lPats = 4.0,
        corr = 0.5,
        cmean = 0.5,
        cvar = 0.1,
        iWeight = rinterface.NULL,
        verbose = False,
    )
    
    Args:
       nItems :  an integer. Number of items to simulate
    
       nTrans :  an integer. Number of transactions to simulate
    
       method :  name of the simulation method used (see Details Section).
    
       ... :  further arguments used for the specific simulation method
      (see details).
    
       verbose :  report progress?
    
     

In [33]:
its_r = R.random_patterns(100, 10)
its_r

<rpy2.robjects.methods.RS4 object at 0x7f63142d3680> [RTYPES.S4SXP]
R classes: ('itemsets',)

Since we directly called a R function, we need to manually wrap the R object as a Python object before we use it in Python.

In [34]:
its_p = Itemsets(its_r)
its_p.as_df()

,items,pWeights,pCorrupts
1,"{item58,item75}",0.315539,0.424112
2,"{item27,item57,item58,item61,item75,item94}",0.012933,0.270126
3,"{item18,item36,item58,item79,item81,item94,ite...",0.221908,0.762388
4,"{item20,item43,item79,item94,item96}",0.003548,0.199264
5,"{item6,item73,item78}",0.167989,0.643824
6,"{item6,item73,item78}",0.000897,0.714733
7,"{item6,item18,item73,item78}",0.028318,0.381193
8,"{item18,item22,item29,item70}",0.096626,0.000000
9,"{item17,item58,item92}",0.061928,1.000000
10,"{item17,item92,item97}",0.090316,0.805467


In [39]:
trans = a2py(R.random_transactions(10, 1000))

print(trans)



transactions in sparse format with
 1000 transactions (rows) and
 10 items (columns)



## Create Rules Objects

To import rules from other tools or to create rules manually, rules for `arules` can be created from lists 
of sets of items. The item labels (i.e., the sparse representation) is
taken from the transactions `trans`.

In [36]:
import rpy2.robjects as ro
from arulespy.arules import Rules, ItemMatrix

trans = Transactions.from_df(pd.read_csv("https://mhahsler.github.io/arulespy/examples/Zoo.csv"))


lhs = [
    ['hair', 'milk', 'predator'],
    ['hair', 'tail', 'predator'],
    ['fins']
]
rhs = [
    ['type=mammal'],
    ['type=mammal'],
    ['type=fish']
]
                          
lhs = ItemMatrix.from_list(lhs, itemLabels = trans)
rhs = ItemMatrix.from_list(rhs, itemLabels = trans)

r = Rules.new(lhs, rhs)
r.as_df()

,LHS,RHS
1,"{hair,milk,predator}",{type=mammal}
2,"{hair,predator,tail}",{type=mammal}
3,{fins},{type=fish}


Next, we add interest measures calculated on the transactions.

In [37]:
r.addQuality(r.interestMeasure(['support', 'confidence', 'lift'], trans))

r.as_df().round(2)

,LHS,RHS,support,confidence,lift
1,"{hair,milk,predator}",{type=mammal},0.20,1.00,2.46
2,"{hair,predator,tail}",{type=mammal},0.16,1.00,2.46
3,{fins},{type=fish},0.13,0.76,5.94
